# 1. Install required packages

In [ ]:
# !pip install numpy
# !pip install pandas
# !pip install matplotlib

# !pip install pyswmm
# !pip install flopy

# 2. Import required packages

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as  plt
import matplotlib.patches as patches
import matplotlib.dates as mdates
from matplotlib.colors import ListedColormap, BoundaryNorm
from datetime import datetime
from shapely.geometry import Polygon

import pyswmm
from pyswmm import Simulation, Subcatchments, Nodes, Links, SimulationPreConfig, Output

from swmm.toolkit.shared_enum import SubcatchAttribute, NodeAttribute, LinkAttribute

import flopy

# 3. SWMM and Pyswmm

## 3.1 Compoenents and structure of Pyswmm

In [ ]:
# check the modules within Pyswmm
pyswmm.__all__

In [ ]:
# check the docstring of the Simulation class
print(Simulation.__doc__)
# Simulation?

### 3.1.1 Sim module

In [ ]:
# Initiliaze the simulation object
sim = Simulation('./Espresso/Example1.inp')

In [ ]:
# attributes, property, methods in the sim object
sim.__dir__()

In [ ]:
# check the property
print(sim.flow_units, sim.start_time, sim.end_time)   # CFS: cubic feet per second

### 3.1.2 Subctachments

In [ ]:
# subcatchements is a collection of subcatchments in the model

subcatchments = Subcatchments(sim)

print(len(subcatchments))
print("1" in subcatchments)

In [ ]:
# Iterate through the subcatchments

for subcatchment in subcatchments:
    print(f'Subcatchment {subcatchment.subcatchmentid} has {subcatchment.area} ha, and its outlet is {subcatchment.connection}')

In [ ]:
# check one subcatchment's property

S1 = subcatchments['1']
print(f'The area of subcatchment {S1.subcatchmentid} is {S1.area} ha')
print(f'The impervious percentage of subcatchment {S1.subcatchmentid} is {S1.percent_impervious}')
print(f'The width of subcatchment {S1.subcatchmentid} is {S1.width}')

### 3.1.3 Nodes: Outfalls, Junctions, divider and storage

#### Practice
* Check how many nodes in the model
* Tell which one is the outfall, others are junctions

### 3.1.4 Links: conduit, orifice, pump, weir and outlet

#### Practice
* Check how many Links in the model
* Tell the type of each Links

### 3.1.5 Run the simulation

In [ ]:
# Running the simulation

sim.execute()

## 3.2 Modifying the input file

### 3.2.1 Modify one input file 

In [ ]:
# using the SimulationPreConfig class to pre-configure the simulation

# Create Config Handle
sim_conf = SimulationPreConfig()

# Specifying the update parameters
# Parameter Order:
# Section, Object ID, Parameter Index, New Value, Obj Row Num (optional)
sim_conf.add_update_by_token("SUBCATCHMENTS", "1", 3, "5")

with Simulation('./Espresso/Example1.inp', sim_preconfig = sim_conf) as sim:
    S1 = Subcatchments(sim)["1"]
    print(S1.area)

### 3.2.2 Batch modification of the input file

In [ ]:
area_increase = {
    "A10%": 1.1,
    "A20%": 1.2,
    "A30%": 1.3,
}

In [ ]:

for key, value in area_increase.items():
    sim_conf = SimulationPreConfig()
    sim_conf.add_update_by_token("SUBCATCHMENTS", "1", 3, f'{10*value}')

    sim_conf.input_files = './Espresso/Example1.inp'
    sim_conf.filename_suffix = f'_{key}'

    with Simulation('./Espresso/Example1.inp', sim_preconfig = sim_conf) as sim:
        S1 = Subcatchments(sim)["1"]
        print(f'{key} area increase: {S1.area}')
        

#### Practice
* Increase the area and width of subcatchment 1 by 10%, 20%, and 30%, respectively

###  3.2.3 Interacting with the simulation

In [ ]:
CONSTANT_FLOWRATE = 10

with Simulation('./Espresso/Example1.inp') as sim:
    node_9 = Nodes(sim)['9']

    for step in sim:
        node_9.generated_inflow(CONSTANT_FLOWRATE)
        
    print(node_9.statistics)

In [ ]:
def flow_rate(time: datetime):
    if time < datetime(1998, 1, 2, 0, 0):
        return 0.0
    else:
        return CONSTANT_FLOWRATE

with Simulation('./Espresso/Example1.inp') as sim:
    node_9 = Nodes(sim)['9']

    for step in sim:
        node_9.generated_inflow(flow_rate(sim.current_time))

    print(node_9.statistics)

## 3.3 Result interpretation

In [ ]:
# Read the output file, store the data in a pandas dataframe

with Output('./Espresso/Example1.out') as out:
    node_head_outfile = out.node_series('21', NodeAttribute.HYDRAULIC_HEAD)
    link_flow_outfile = out.link_series('15', LinkAttribute.FLOW_RATE)

In [ ]:
node_head_outfile

In [ ]:

fig = plt.figure(figsize=(8,4), dpi=200) #Inches Width, Height
fig.suptitle("Node 21 Head and Link 15 Flow from output")

# Plot from the results compiled during simulation time
# add the first panel to the figure
axis_1 = fig.add_subplot(2,1,1)   

# Plot node head from the output file
x = node_head_outfile.keys() 
y = [node_head_outfile[key] for key in node_head_outfile.keys()]
axis_1.plot(x, y)
axis_1.set_ylabel("Head (ft)")
axis_1.grid("xy")

# add the second panel to the figure, shared the x-axis with the first panel
axis_2 = fig.add_subplot(2,1,2, sharex=axis_1)

# plot the link flowrate from the output file
x = link_flow_outfile.keys()
y = [link_flow_outfile[key] for key in link_flow_outfile.keys()]
axis_2.plot(x, y)
axis_2.set_ylabel("Flow (CFS)")
axis_2.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d %Hh'))
axis_2.grid("xy")

fig.autofmt_xdate()
plt.tight_layout()
plt.savefig("TEST.PNG")
plt.show()


# 4. Modflow and Flopy

## 4.1 Create a basic MODFLOW model object
As well as any other model generated by Flopy, first we have to setup the model name and the working directory. We strongly recommend to follow the folder/file estructure provided on the "Input Files" of this tutorial. Based on the complex geometry and the differences among layers elevations and layer thickness, the NWT solver has been applied to the model.

In [ ]:
modelname = "model1"
modelpath = "./Model/"
#Modflow Nwt
mf1 = flopy.modflow.Modflow(modelname, exe_name="./Exe/MODFLOW-NWT_64.exe", version="mfnwt",model_ws=modelpath)
nwt = flopy.modflow.ModflowNwt(mf1 ,maxiterout=150,linmeth=2)

## 4.2 Spatial Discretization parameter

In [ ]:
# Initialize the 20x20 grid
dem_data = np.zeros((20, 20))

# Set the first and last column values
for i in range(20):
    dem_data[:,i] = 100 - i

print(dem_data)

In [ ]:
# the layer vertcial descritization 

Layer1 = dem_data -10
Layer2 = dem_data -50
Layer3 = 0

In [ ]:
# the horizontal discretization: column and row

ztop = dem_data
zbot = [Layer1, Layer2, Layer3]

nlay = 3

nrow = 20
ncol = 20
delr = 500
delc = 500

## 4.3 Definition of the flow packages for the MODFLOW model

In this part we define the packages related to groundwater flow on the MODFLOW model. First we define the DIS package that has the geometry as well as the simulation type (steady / transient). The model run on steady conditions.

In [ ]:
dis = flopy.modflow.ModflowDis(mf1, nlay,nrow,ncol,delr=delr,delc=delc,top=ztop,botm=zbot,itmuni=1, steady=True)

Then we define another MODEL packages. Some boundary conditions are setup from raster valures, while other packages use constant value. 
These are the required packages for the simulation:
- the BAS package for setting the constant head cells and inactive cells,
- the UPW that defines the vertical / horizontal hydraulic conductivity,
- the RCH package that applies rechage from precipitation
- the EVT package for the simulation of groundwater consumption from roots
- the DRN package for the groundwater discharge to the channel network
- the OC packages for the output record

In [ ]:
# Variables for the BAS package
iboundData = np.zeros(dem_data.shape, dtype=np.int32)
iboundData = 1  # 1 means active cell
bas = flopy.modflow.ModflowBas(mf1,ibound=iboundData,strt=ztop-50, hnoflo=-2.0E+020)

In [ ]:
# Array of hydraulic heads per layer
hk = [1E-3, 1E-4, 1E-5]

# Add UPW package to the MODFLOW model
upw = flopy.modflow.ModflowUpw(mf1, laytyp = [1,1,0], hk = hk)

In [ ]:
#Add the recharge package (RCH) to the MODFLOW model
rch = np.ones((nrow, ncol), dtype=np.float32) * 0.1/365/86400
rch_data = {0: rch}
rch = flopy.modflow.ModflowRch(mf1, nrchop=3, rech =rch_data)

In [ ]:
# the constant-boundary condition
chd_array = np.zeros(dem_data.shape, dtype=np.int32)
chd_array[:,1]= 1
chd_array[:,-1]= 1

lst = []
for k in range(nlay):
    for i in range(chd_array.shape[0]):
        for q in range(chd_array.shape[1]):
            if chd_array[i,q] == 1:
                elevation = ztop[i,q] - 50
                lst.append([k,i,q,elevation,elevation]) #layer,row,column, starting head, ending head
chd_spd = {0:lst}
chd = flopy.modflow.ModflowChd(mf1, stress_period_data=chd_spd)

In [ ]:
# Add OC package to the MODFLOW model
oc = flopy.modflow.ModflowOc(mf1) #ihedfm= 1, iddnfm=1

## 4.4 Model Run

In [ ]:
#Write input files -> write file with extensions
mf1.write_input()

#run model -> gives the solution
mf1.run_model()

## 4.5 Model results post-processing

This tutorial has some model features and output representation done with Flopy and Matplotlib. Another 3D representation on the VTK format is performed on another notebook from this tutorial

In [ ]:
#Import model
ml = flopy.modflow.Modflow.load('./Model/model1.nam')

**Model grid representation**

In [ ]:
# First step is to set up the plot
fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(1, 1, 1, aspect='equal')

# Next we create an instance of the ModelMap class
modelmap = flopy.plot.PlotMapView(ml)

# Then we can use the plot_grid() method to draw the grid
# The return value for this function is a matplotlib LineCollection object,
# which could be manipulated (or used) later if necessary.
linecollection = modelmap.plot_grid(linewidth=0.4)

**Cross section of model grid representation**

In [ ]:
hdobj = flopy.utils.HeadFile('./Model/model1.hds')
head = hdobj.get_data()

fig = plt.figure(figsize=(30, 30))
ax = fig.add_subplot(1, 2, 1, aspect='equal')
modelmap = flopy.plot.PlotMapView(model=ml)
# modelmap.plot_bc('DRN', color='red')
#quadmesh = modelmap.plot_ibound()
quadmesh = modelmap.plot_array(head, alpha=0.8)
linecollection = modelmap.plot_grid(linewidth=0.2)
plt.colorbar(quadmesh, ax=ax, shrink=0.5)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 6))
ax = fig.add_subplot(1, 1, 1)
# Next we create an instance of the ModelCrossSection class
#modelxsect = flopy.plot.ModelCrossSection(model=ml, line={'Column': 5})
modelxsect = flopy.plot.PlotCrossSection(model=ml, line={'Row': 10})

# Then we can use the plot_grid() method to draw the grid
# The return value for this function is a matplotlib LineCollection object,
# which could be manipulated (or used) later if necessary.
linecollection = modelxsect.plot_grid(linewidth=0.4)
linecollection = modelxsect.plot_surface(head,ax=ax)
t = ax.set_title('Column 6 Cross-Section - Model Grid')

# 5. Mapping the SWMM to Modflow

## 5.1 Extract Polygons of subcatchments

In [ ]:
import re

data = open('./Espresso/Example1.inp').read()

polygons_data = re.findall(r"\[Polygons\](.*?)\[", data, re.DOTALL)[0]

# Match each line of polygon data
pattern = r"(\d+)\s+([\d\.]+)\s+([\d\.]+)"
matches = re.findall(pattern, polygons_data)

polygons = {}
for match in matches:
    subcatchment, x_coord, y_coord = match
    if subcatchment not in polygons:
        polygons[subcatchment] = []
    polygons[subcatchment].append((float(x_coord), float(y_coord)))

# Ensure polygons are closed by repeating the first point at the end if necessary
# for subcatchment, coords in polygons.items():
#     if coords[0] != coords[-1]:  # Check if first and last points are the same
#         coords.append(coords[0])  # Make polygon closed by adding the first point at the end

for subcatchment, coords in polygons.items():
    print(f"Subcatchment {subcatchment}:")
    for coord in coords:
        print(f"  X-Coord: {coord[0]}, Y-Coord: {coord[1]}")

In [ ]:
# draw the ploygon:
fig, ax = plt.subplots(figsize=(10, 10))
for subcatchment, coords in polygons.items():
    x, y = zip(*coords)
    ax.plot(x, y, label=f"Subcatchment {subcatchment}")

# plot a square 10000 ft by 10000 ft
square = patches.Rectangle((0,0), 9500, 10000, linewidth=1, edgecolor='r', facecolor='none')
ax.add_patch(square)

ax.set_xticks(range(0, 10000, 500))
ax.set_yticks(range(0, 10000, 500))
ax.grid(True)

ax.legend(ncol=3, bbox_to_anchor=(0.7, -0.05))
plt.show()

In [ ]:
polygons

## 5.2 Mapping the cell to the subcatchment

In [ ]:
mapping_matrix = np.zeros((20, 20))

for i in range(20):
    for j in range(20):
        cell = Polygon(
            [
                (j * 500, (20 - i) * 500),
                (j * 500 + 500, (20 - i) * 500),
                (j * 500 + 500, (20 - i) * 500 - 500),
                (j * 500, (20 - i) * 500 - 500)  # Corrected this line
            ]
        )

        found = False  # Flag to check if a subcatchment contains the cell
        for subcatchment, coords in polygons.items():
            polygon = Polygon(coords)
            if polygon.contains(cell):
                mapping_matrix[i, j] = int(subcatchment)
                found = True
                break  # Exit the loop once a containing polygon is found
        if not found:
            mapping_matrix[i, j] = 0  # Assign 0 if no containing polygon is found

In [ ]:
# Use a default colormap (e.g., 'viridis', 'plasma', 'inferno', 'magma', 'cividis')
cmap = plt.get_cmap('viridis')

# Define boundaries for the colors
bounds = np.arange(-0.5, 9, 1)

# Create a BoundaryNorm instance
norm = BoundaryNorm(bounds, cmap.N)

fig, ax = plt.subplots(figsize=(10, 10))
cax = ax.matshow(mapping_matrix, cmap=cmap, norm=norm)

# Add colorbar with ticks at the center of each segment
fig.colorbar(cax, ticks=np.arange(0, 9))

# Annotate each cell with the numeric value
for i in range(mapping_matrix.shape[0]):
    for j in range(mapping_matrix.shape[1]):
        text = ax.text(j, i, mapping_matrix[i, j],
                       ha="center", va="center", color="w")


plt.show()


# 6. Coupling SWMM and Modflow

## 6.1 Change the Modflow modle to transient study

In [ ]:
dis = flopy.modflow.ModflowDis(mf1, nlay,nrow,ncol,delr=delr,delc=delc,top=ztop,botm=zbot,itmuni=1, steady=False)

In [ ]:
strt = ztop - 50

## 6.1 Coupling

In [ ]:
with Simulation('./Espresso/Example2.inp') as sim:
    step_counter = 0
    hourly_counter = 0
    infil_dict_current_dt = {}  # Initialize current timestep infiltration dictionary
    infil_dict_previous_acc = {}  # Initialize previous accumulated infiltration dictionary

    for step in sim:
        step_counter += 1

        if step_counter == 60:  # Every 60 steps, assuming 1 step = 1 minute for hourly processing
            step_counter = 0

            print('Current Time:', sim.current_time)

            subcatchments = Subcatchments(sim)

            # Process infiltration rates
            for subcatchment in subcatchments:
                infiltration = subcatchment.statistics['infiltration']
                sub_id = subcatchment.subcatchmentid
                if sub_id not in infil_dict_previous_acc:
                    infil_dict_previous_acc[sub_id] = 0  # Initialize if not present
                infil_rate = (infiltration - infil_dict_previous_acc[sub_id]) / (subcatchment.area * 1000)
                infil_dict_current_dt[sub_id] = infil_rate
                infil_dict_previous_acc[sub_id] = infiltration

            # Prepare recharge array
            recharge_array = np.zeros((nrow, ncol))
            for i in range(nrow):
                for j in range(ncol):
                    if mapping_matrix[i, j] != 0:
                        recharge_array[i, j] = infil_dict_current_dt.get(str(int(mapping_matrix[i, j])), 0)

            rch_data = {0: recharge_array}
            rch = flopy.modflow.ModflowRch(mf1, nrchop=3, rech=rch_data)
            bas = flopy.modflow.ModflowBas(mf1, ibound=iboundData, strt=strt)

            mf1.write_input()

            try:
                success, buff = mf1.run_model(silent=True)
                if not success:
                    raise Exception("MODFLOW run failed.")
            except Exception as e:
                print(f"Error in MODFLOW: {e}")
                break

            fname = os.path.join('Model', 'model1.hds')
            headfile = flopy.utils.HeadFile(fname, model=mf1)
            heads = headfile.get_data()
            heads[np.isin(heads, [1.e+30, -999.99])] = np.nan  # Replace invalid values with NaN

            strt = heads[0]  # Update starting heads for the next simulation step

## 6.3 Visualizing the Results

In [ ]:
hdobj = flopy.utils.HeadFile('./Model/model1.hds')
head = hdobj.get_data()

fig = plt.figure(figsize=(30, 30))
ax = fig.add_subplot(1, 2, 1, aspect='equal')
modelmap = flopy.plot.PlotMapView(model=mf1)
# modelmap.plot_bc('DRN', color='red')
#quadmesh = modelmap.plot_ibound()
quadmesh = modelmap.plot_array(head, alpha=0.8)
linecollection = modelmap.plot_grid(linewidth=0.2)
plt.colorbar(quadmesh, ax=ax, shrink=0.5)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 6))
ax = fig.add_subplot(1, 1, 1)
# Next we create an instance of the ModelCrossSection class
#modelxsect = flopy.plot.ModelCrossSection(model=ml, line={'Column': 5})
modelxsect = flopy.plot.PlotCrossSection(model=mf1, line={'Row': 10})

# Then we can use the plot_grid() method to draw the grid
# The return value for this function is a matplotlib LineCollection object,
# which could be manipulated (or used) later if necessary.
linecollection = modelxsect.plot_grid(linewidth=0.4)
linecollection = modelxsect.plot_surface(head,ax=ax)
t = ax.set_title('Column 6 Cross-Section - Model Grid')